In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Business Problems

For example:
* How should we allocate our limited marketing budget for next year?
* What type on influencers should we focus on?
* Can we cut budget from TV, as it is too expensive?
* Finally, set your assumptions of the marketing budget and influencer (if any). Can you try to predict the expected sales based on the best ML model?

# Importing the Libraries
First we import the required libraries:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

# Reading the Data

In [ ]:
df = pd.read_csv('/kaggle/input/dummy-advertising-and-sales-data/Dummy Data HSS.csv')
df.head()

We want to look if there's any missing data, as follows:

In [ ]:
df.info()

We can see there are mising data in several columns. Thus, we want to fill the missing data with its average, as follows:

In [ ]:
df = df.fillna(df.mean()) # updates the df

# see the updated df info

df.info()

Now we have a dataframe that has no missing values. Now, we want to encode the categorical variable to dummy variables, as follows:

In [ ]:
df = pd.get_dummies(df) # updates the df again

# see the updated df

df

Now for the purpose of simplicity, we shift the column 'Sales' to the end of the table, as follows:

In [ ]:
df.columns # getting the column names

In [ ]:
df = df[['TV', 'Radio', 'Social Media', 'Influencer_Macro',
       'Influencer_Mega', 'Influencer_Micro', 'Influencer_Nano', 'Sales']]

# see the updated df

df.head()

Out of curiosity, we just want to explore whether there is any correlation of Sales with its predictors:

In [ ]:
df.corr()

Now we have a clean data. Now, we will create the independent and dependent variables (x and y), as follows:

In [ ]:
x = df.iloc[:,0:-1].values
y = df.iloc[:,-1:].values

Let's see some preview of x and y:

In [ ]:
print(x)
print(y)

Looks good. Now, let's split our data for training and testing:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

Let's see some previews:

In [ ]:
print(x_train)
print(y_train)

We want to know the number of data used for training and testing, as follows:

In [ ]:
print(len(x_train),len(x_test))

# Analysis

Now we will train and predict the data based on several regression models:
* Linear
* Random Forest
* Decision Tree
* Support Vector
* Polynomial

For each regression model, we will evaluate its r2_score and root mean squared error (RMSE). The higher r2_score the better; the lower RMSE, the better.

## Linear Regression

In [ ]:
lr_regressor = LinearRegression() # instantiate the Linear Regression module
lr_regressor.fit(x_train, y_train) # training the data

# after training the data, perform prediction:

y_pred_lr = lr_regressor.predict(x_test) # this is the prediction

# evaluate the r2_score and RMSE between prediction and real data

print(r2_score(y_test, y_pred_lr))
print(mse(y_test, y_pred_lr)**0.5)

In [ ]:
print(lr_regressor.coef_, lr_regressor.intercept_)

In [ ]:
df.columns

Linear Regression model shows that:

**Sales = 3.50 TV + 0.14 Radio + 0.03 Social Media - 0.43 Macro + 0.13 Mega + 0.07 Micro + 0.22 Nano**

## Random Forest Regression

In [ ]:
rf_regressor = RandomForestRegressor() # instantiate the Random Forest Regression module
rf_regressor.fit(x_train, y_train) # training the data

# after training the data, perform prediction:

y_pred_rf = rf_regressor.predict(x_test) # prediction data

# evaluate the r2_score and RMSE between prediction and real data

print(r2_score(y_test, y_pred_rf))
print(mse(y_test, y_pred_rf)**0.5)

## Decision Tree Regression

In [ ]:
# Similar processes with the above two examples

dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(x_train, y_train)

y_pred_dt = dt_regressor.predict(x_test)

print(r2_score(y_test, y_pred_dt))
print(mse(y_test, y_pred_dt)**0.5)

## Support Vector Regression

For SVR, we need to perform feature scaling. In short, this is required because of some mathematical assumptions. So first we will perform feature scaling:

### Feature Scaling for Support Vector Regression

To avoid confusion of variable names, we will create new independent and dependent variable names: a and b

In [ ]:
a = df.iloc[:,0:-1].values
b = df.iloc[:,-1:].values

# reshape b

b = b.reshape(len(b), 1)

In [ ]:
# now we split to train and test

a_train, a_test, b_train, b_test = train_test_split(a, b)

In [ ]:
# now perform scaling

scala = StandardScaler()
scalb = StandardScaler()

a_train = scala.fit_transform(a_train)
b_train = scalb.fit_transform(b_train)

In [ ]:
# training the SVR

sv_regressor = SVR()
sv_regressor.fit(a_train, b_train)

In [ ]:
# create the prediction

b_pred = scalb.inverse_transform(sv_regressor.predict(scala.transform(a_test)))

In [ ]:
# evaluate the model

print(r2_score(b_test, b_pred))
print(mse(b_test, b_pred)**0.5)

### Polynomial Regression

For Polynomial Regression, we also have to perform polynomial feature scaling.

In [ ]:
poly = PolynomialFeatures()
po_regressor = LinearRegression()

# training the data

po_regressor.fit(poly.fit_transform(x_train), y_train)

In [ ]:
# prediction training

y_pred_po = po_regressor.predict(poly.fit_transform(x_test))

# evaluate the model

print(r2_score(y_test, y_pred_po))
print(mse(y_test, y_pred_po)**0.5)

## Model Evaluation

* Based on our analysis, it can be inferred that Support Vector Regression yields the highest R2 score and lowest RMSE. Thus, SVR will be the chosen model. We can use SVR model to predict the sales based on the input strategy.
* Based on linear regression, we can infer that we can focus on TV, Radio, and Nano Influencer. We can cut budget related to other factors.

# Concluding Remarks

* I want this notebook to be as understandable as possible so students or people like me who are not from math/data science/computer science background can infer easily what is happening in each line of code.
* I understand there will be several math/ML/any other assumptions that I might have skipped in this notebook (e.g. assumptions for Linear Regression, feature scaling, parameter tuning). Please let me know in the comments to improve it.
* Overall, this is part of my learning journey as well as a business management scholar. So, I will just focus on the 'broad overview' of the algorithms, and shed a light more on *making sense* of how data can be used for better decision making

You can upvote if you like this notebook. Happy (machine) learning!